In [1]:
import uuid, sys, os
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
import math
import random

import torch
print("PyTorch:", torch.__version__)

from tqdm import tqdm
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print("Current location:", os.getcwd())

# from esm.models.esmc import ESMC
# from esm.sdk.api import ESMProtein, LogitsConfig

import torch
print(torch.version.cuda)   # shows the CUDA version PyTorch was built against
print(torch.backends.cudnn.version())  # cuDNN version
print(torch.cuda.is_available())
import training_utils.partitioning_utils as pat_utils

/work3/s232958/envs/esm_inverse/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch: 1.12.1.post201
Using device: cuda
Current location: /zhome/c9/0/203261/DBL046_PP_osaul/DBL046_PP_osaul/tmp/ona_drafts
11.2
8907
True


In [2]:
path_to_mmseqs_clustering = "/work3/s232958/data/PPint_DB/3_å_dataset5_singlefasta/clusterRes40"
all_seqs, clust, clust_keys = pat_utils.mmseqs_parser(path_to_mmseqs_clustering)

path_to_interaction_df = "/work3/s232958/data/PPint_DB/disordered_interfaces_no_cutoff_filtered_nonredundant80_3å_5.csv.gz"
disordered_interfaces_df = pd.read_csv(path_to_interaction_df,index_col=0).reset_index(drop=True)
disordered_interfaces_df["PDB_chain_name"] = (disordered_interfaces_df["PDB"] + "_" + disordered_interfaces_df["chainname"]).tolist()
disordered_interfaces_df["index_num"] = np.arange(len(disordered_interfaces_df))
disordered_interfaces_df["chain_name_index"] = [row["PDB_chain_name"] + "_" + str(row["index_num"]) for index, row in disordered_interfaces_df.iterrows()]
disordered_interfaces_df = disordered_interfaces_df.set_index("PDB_interface_name")
disordered_interfaces_df["interface_residues"] = disordered_interfaces_df["interface_residues"].apply(lambda x: ast.literal_eval(x))
# disordered_interfaces_df["inter_chain_hamming"] = [1 - (Ldistance(seq.split("-")[0], seq.split("-")[1]))/np.max([len(seq.split("-")[0]), len(seq.split("-")[1])]) for seq in disordered_interfaces_df["protien_interface_sequences"]]
disordered_interfaces_df["dimer"] = disordered_interfaces_df["inter_chain_hamming"] > 0.60
disordered_interfaces_df["clust_keys"] = [clust_keys.get(row["chain_name_index"]) for index, row in disordered_interfaces_df.iterrows()] 
print(disordered_interfaces_df["clust_keys"].head())

pdb_interface_and_clust_keys = {index:disordered_interfaces_df.loc[index,"clust_keys"].values.tolist() for index in tqdm(disordered_interfaces_df.index.drop_duplicates(), total=len(disordered_interfaces_df)/2)}
new_clusters, new_clusters_clustkeys = pat_utils.recluster_mmseqs_keys_to_non_overlapping_groups(pdb_interface_and_clust_keys)

### Creating train and test datasets based on train and test-idexes
train_indexes, test_indexes = pat_utils.run_train_test_partition(interaction_df=disordered_interfaces_df,
                                                    clustering=new_clusters, # Clusters from Bidentate-graphs
                                                    train_ratio=0.8, 
                                                    test_ratio=0.2, 
                                                    v=True, 
                                                    seed=0)

disordered_interfaces_df["ID"] = [row["PDB"]+"_"+str(row["interface_index"])+"_"+row["chainname"] for __, row in disordered_interfaces_df.iterrows()]
disordered_interfaces_df["PDB_interface_name"] = disordered_interfaces_df.index

PDB_interface_name
6NZA_0    6NZA_B_1
6NZA_0    6NZA_B_1
9JKA_1    9JKA_C_3
9JKA_1    9JKA_C_3
8DQ6_1    8DQ6_B_4
Name: clust_keys, dtype: object


100%|██████████████████████████████████████████████████████████████████████████| 24725/24725.0 [00:37<00:00, 665.34it/s]
100%|█████████████████████████████████████████████████████████████████████████| 27834/27834 [00:00<00:00, 856197.62it/s]


0.8
0.2


In [3]:
print(f"Train indeces number: {len(train_indexes)}")
print(f"Test indeces number: {len(test_indexes)}")

Train indeces number: 19781
Test indeces number: 4944


In [4]:
disordered_interfaces_df

,PDB,interface_index,chainname,sequence,interface_residues,disorderscore_predictions,mean disorder interface,seq. len,protien_interface_sequences,interacting_seq_aa,inter_chain_hamming,PDB_chain_name,index_num,chain_name_index,dimer,clust_keys,ID,PDB_interface_name
PDB_interface_name,,,,,,,,,,,,,,,,,,
6NZA_0,6NZA,0,A,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,"[184, 185, 186, 187, 188, 191, 192, 194, 198, ...","[0.7864, 0.7974, 0.7796, 0.7675, 0.7414, 0.720...",0.038646,461,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,GRTHLATLQESGWAMHYAVTGLNFEDVQGAGASMKNDMVNGASGNF...,0.995662,6NZA_A,0,6NZA_A_0,True,6NZA_B_1,6NZA_0_A,6NZA_0
6NZA_0,6NZA,0,B,TVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHAL...,"[182, 183, 184, 185, 186, 190, 192, 196, 197, ...","[0.6701, 0.7181, 0.6893, 0.671, 0.6561, 0.6428...",0.037456,459,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,GRTHLATLQESGWAMHYAVTGLNFEDVQGAGASMKNDMVNGASGNF...,0.995662,6NZA_B,1,6NZA_B_1,True,6NZA_B_1,6NZA_0_B,6NZA_0
9JKA_1,9JKA,1,B,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[34, 35, 38, 39, 42, 45, 46, 49, 53, 56, 57, 6...","[0.3113, 0.3238, 0.2933, 0.2873, 0.2903, 0.289...",0.071228,362,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,TAGLSFKIGALKASLPVEHHERKARVASMHDMTKAFLEAKQWTLVS...,1.000000,9JKA_B,2,9JKA_B_2,True,9JKA_C_3,9JKA_1_B,9JKA_1
9JKA_1,9JKA,1,C,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[83, 94, 105, 106, 108, 109, 112, 113, 116, 11...","[0.3113, 0.3238, 0.2933, 0.2873, 0.2903, 0.289...",0.097310,362,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,TAGLSFKIGALKASLPVEHHERKARVASMHDMTKAFLEAKQWTLVS...,1.000000,9JKA_C,3,9JKA_C_3,True,9JKA_C_3,9JKA_1_C,9JKA_1
8DQ6_1,8DQ6,1,B,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,"[15, 19, 20, 23, 34, 35, 36, 37, 38, 39, 40, 4...","[0.1487, 0.1646, 0.1496, 0.1519, 0.1521, 0.155...",0.037771,109,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,CLKTESYVMILLNFFGY-VPIAFAGGSESSSRFY,0.889908,8DQ6_B,4,8DQ6_B_4,True,8DQ6_B_4,8DQ6_1_B,8DQ6_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7OXG_0,7OXG,0,C,TRYWNAKALPFAFG,"[0, 1, 4, 5, 8, 9, 10, 11, 12]","[0.7407, 0.7988, 0.784, 0.794, 0.8071, 0.832, ...",0.825889,14,MKVGQDKVVTIRYTLQVEGEVLDQGELSYLHGHRNLIPGLEEALEG...,LDRNLIAYG-TRNALPFAF,0.111111,7OXG_C,49445,7OXG_C_49445,False,7OXG_C_49445,7OXG_0_C,7OXG_0
6UUM_0,6UUM,0,F,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,"[43, 44, 45, 46, 94, 95, 96, 97, 98, 99, 100, ...","[0.3132, 0.2896, 0.2667, 0.2557, 0.2512, 0.253...",0.143800,215,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,VPRLSFFTFGPGFIFPSEQSVTSSKSFEC-GLEWKGSDWPGFPLAK...,0.264317,6UUM_F,49446,6UUM_F_49446,False,5I19_L_30828,6UUM_0_F,6UUM_0
6UUM_0,6UUM,0,H,QVQLVQSGAEVRKPGSSVTISCKPVGGTFTNFAIHWVRQAPGQGLE...,"[43, 44, 45, 46, 58, 60, 110, 111, 113, 115, 1...","[0.3256, 0.3418, 0.3422, 0.3367, 0.3317, 0.314...",0.168272,227,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,VPRLSFFTFGPGFIFPSEQSVTSSKSFEC-GLEWKGSDWPGFPLAK...,0.264317,6UUM_H,49447,6UUM_H_49447,False,7U0E_A_27374,6UUM_0_H,6UUM_0


In [5]:
# Creating new dataframe with pairs of proteins (PPints)
grouped = {}
for _, row in disordered_interfaces_df.iterrows():
    iface = row["PDB_interface_name"]
    seq = row["sequence"]
    rid = row["ID"]
    dimer = row["dimer"]
    inter_res = row["interface_residues"]
    inter_seq = row["protien_interface_sequences"]
    
    if iface not in grouped:
        grouped[iface] = {
            "sequences": [],
            "IDs": [],
            "interface_residues" : [],
            "interface_sequences" : [],
            "dimer": dimer,
        }
    else:
        if grouped[iface]["dimer"] != dimer:
            print(f"Warning: multiple dimers for interface {iface}:",
                  grouped[iface]['dimer'], "vs", dimer)

    grouped[iface]["sequences"].append(seq)
    grouped[iface]["interface_residues"].append(inter_res)
    grouped[iface]["interface_sequences"].append(inter_seq)
    grouped[iface]["IDs"].append(rid)

records = []
for iface, vals in grouped.items():
    seqs = vals["sequences"]
    ids = vals["IDs"]
    inter_res = vals["interface_residues"]
    inter_seq = vals["interface_sequences"]
    if len(seqs) >= 2 and len(ids) >= 2:
        records.append({
            "interface_id": iface,
            "ID1": ids[0],
            "seq_target": seqs[0],
            "inter_res_target" : inter_res[0],
            "inter_seq_target" : inter_seq[0],

            "ID2": ids[1],
            "seq_binder": seqs[1],
            "inter_res_binder" : inter_res[1],
            "inter_seq_binder" : inter_seq[1],
            "dimer": vals["dimer"],
        })

PPint_interactions_NEW = pd.DataFrame(records)
PPint_interactions_NEW["seq_target_len"] = [len(row.seq_target) for __, row in PPint_interactions_NEW.iterrows()]
PPint_interactions_NEW["seq_binder_len"] = [len(row.seq_binder) for __, row in PPint_interactions_NEW.iterrows()]
PPint_interactions_NEW["target_binder_id"] = PPint_interactions_NEW["ID1"] + "_" + PPint_interactions_NEW["ID2"]

PPint_interactions_NEW

,interface_id,ID1,seq_target,inter_res_target,inter_seq_target,ID2,seq_binder,inter_res_binder,inter_seq_binder,dimer,seq_target_len,seq_binder_len,target_binder_id
0,6NZA_0,6NZA_0_A,MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,"[184, 185, 186, 187, 188, 191, 192, 194, 198, ...",MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,6NZA_0_B,TVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIHAL...,"[182, 183, 184, 185, 186, 190, 192, 196, 197, ...",MNTVRSEKDSMGAIDVPADKLWGAQTQRSLEHFRISTEKMPTSLIH...,True,461,459,6NZA_0_A_6NZA_0_B
1,9JKA_1,9JKA_1_B,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[34, 35, 38, 39, 42, 45, 46, 49, 53, 56, 57, 6...",VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,9JKA_1_C,VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,"[83, 94, 105, 106, 108, 109, 112, 113, 116, 11...",VAAGATLALLSFLTPLAFLLLPPLLWREELEPCGTACEGLFISVAF...,True,362,362,9JKA_1_B_9JKA_1_C
2,8DQ6_1,8DQ6_1_B,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,"[15, 19, 20, 23, 34, 35, 36, 37, 38, 39, 40, 4...",PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,8DQ6_1_C,PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,"[45, 46, 47, 48, 49, 50, 51, 70, 73, 74, 77, 8...",PTLNLFTNIPVDAVTCSDILKDATKAVAKIIGKPESYVMILLNSGV...,True,109,97,8DQ6_1_B_8DQ6_1_C
3,2YMZ_0,2YMZ_0_A,ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,"[1, 2, 3, 4, 5, 6, 7, 8, 122, 124, 125, 126, 1...",ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,2YMZ_0_B,ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,"[1, 2, 3, 4, 5, 6, 7, 8, 122, 124, 125, 126, 1...",ARMFEMFNLDWKSGGTMKIKGHISEDAESFAINLGCKSSDLALHFN...,True,130,130,2YMZ_0_A_2YMZ_0_B
4,6IDB_0,6IDB_0_A,DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18,...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,6IDB_0_B,GLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAID...,"[5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, ...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,False,317,172,6IDB_0_A_6IDB_0_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24720,6O42_0,6O42_0_L,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,"[43, 45, 46, 91, 92, 93, 94, 95, 96, 97, 98, 1...",EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,6O42_0_H,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSSYAISWVRQAPGQGLE...,"[43, 44, 45, 46, 60, 61, 110, 111, 113, 114, 1...",EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPR...,False,214,220,6O42_0_L_6O42_0_H
24721,2QIB_0,2QIB_0_A,GVEERRQQLIGVALDLFSRRSPDEVSIDEIASAAGISRPLVYHYFP...,"[102, 103, 104, 135, 136, 139, 143, 144, 146, ...",GVEERRQQLIGVALDLFSRRSPDEVSIDEIASAAGISRPLVYHYFP...,2QIB_0_B,RRQQLIGVALDLFSRRSPDEVSIDEIASAAGISRPLVYHYFPGKLS...,"[95, 96, 97, 100, 125, 126, 129, 133, 134, 136...",GVEERRQQLIGVALDLFSRRSPDEVSIDEIASAAGISRPLVYHYFP...,True,217,204,2QIB_0_A_2QIB_0_B
24722,7OXG_0,7OXG_0_A,MKVGQDKVVTIRYTLQVEGEVLDQGELSYLHGHRNLIPGLEEALEG...,"[21, 22, 33, 34, 35, 36, 61, 62, 63]",MKVGQDKVVTIRYTLQVEGEVLDQGELSYLHGHRNLIPGLEEALEG...,7OXG_0_C,TRYWNAKALPFAFG,"[0, 1, 4, 5, 8, 9, 10, 11, 12]",MKVGQDKVVTIRYTLQVEGEVLDQGELSYLHGHRNLIPGLEEALEG...,False,99,14,7OXG_0_A_7OXG_0_C
24723,6UUM_0,6UUM_0_F,EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,"[43, 44, 45, 46, 94, 95, 96, 97, 98, 99, 100, ...",EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,6UUM_0_H,QVQLVQSGAEVRKPGSSVTISCKPVGGTFTNFAIHWVRQAPGQGLE...,"[43, 44, 45, 46, 58, 60, 110, 111, 113, 115, 1...",EIVLTQSPVTLSLSSGETGTLSCRASQNISSSWIAWYQQRRGQVPR...,False,215,227,6UUM_0_F_6UUM_0_H


In [6]:
PPint_interactions_NEW.to_csv("/work3/s232958/data/PPint_DB/PPInt_full_Df_before_split.csv")

In [13]:
# sample random 10%
random.seed(0)
train_indexes_sample = random.sample(train_indexes, int(len(train_indexes) * 0.1))
test_indexes_sample = random.sample(test_indexes, int(len(test_indexes) * 0.1))
# train_indexes_sample = random.sample(train_indexes, int(len(train_indexes)))
# test_indexes_sample = random.sample(test_indexes, int(len(test_indexes)))

Df_train = PPint_interactions_NEW[PPint_interactions_NEW.interface_id.isin(train_indexes_sample)]
Df_test = PPint_interactions_NEW[PPint_interactions_NEW.interface_id.isin(test_indexes_sample)]

In [14]:
Df_train

,interface_id,ID1,seq_target,inter_res_target,inter_seq_target,ID2,seq_binder,inter_res_binder,inter_seq_binder,dimer,seq_target_len,seq_binder_len,target_binder_id
4,6IDB_0,6IDB_0_A,DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18,...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,6IDB_0_B,GLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAID...,"[5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, ...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,False,317,172,6IDB_0_A_6IDB_0_B
8,2WZP_3,2WZP_3_D,VQLQESGGGLVQAGGSLRLSCTASRRTGSNWCMGWFRQLAGKEPEL...,"[37, 38, 39, 40, 41, 86, 87, 89, 90, 91, 116, ...",TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,2WZP_3_G,TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,"[164, 165, 166, 167, 203, 204, 205, 206, 207, ...",TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,False,122,266,2WZP_3_D_2WZP_3_G
11,1ZKP_0,1ZKP_0_A,LYFQSNAKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLA...,"[15, 16, 17, 18, 19, 20, 21, 22, 41, 42, 44, 4...",AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,1ZKP_0_C,AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,"[9, 10, 11, 12, 13, 14, 15, 16, 35, 36, 38, 39...",AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,True,246,240,1ZKP_0_A_1ZKP_0_C
25,6GRH_3,6GRH_3_C,SKHELSLVEVTHYTDPEVLAIVKDFHVRGNFASLPEFAERTFVSAV...,"[3, 5, 6, 7, 10, 11, 13, 14, 15, 246, 247, 248...",MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,6GRH_3_D,MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,"[0, 1, 2, 16, 18, 19, 20, 21, 22, 23, 24, 270,...",MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,False,266,396,6GRH_3_C_6GRH_3_D
35,8R57_1,8R57_1_M,DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,"[22, 23, 26, 27, 30, 31, 47, 48, 50, 51, 54, 5...",DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,8R57_1_f,PKKQKHKHKKVKLAVLQFYKVDDATGKVTRLRKECPNADCGAGTFM...,"[12, 13, 14, 15, 18, 20, 25, 26, 27, 40, 41, 4...",DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,False,118,64,8R57_1_M_8R57_1_f
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24684,4YO8_0,4YO8_0_A,HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,"[38, 39, 58, 59, 61, 62, 65, 66, 69, 70, 73, 7...",HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,4YO8_0_B,HHHHHENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNV...,"[42, 43, 62, 63, 65, 66, 69, 70, 73, 74, 77, 7...",HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,True,238,242,4YO8_0_A_4YO8_0_B
24688,3CKI_0,3CKI_0_A,DPMKNTCKLLVVADHRFYRYMGRGEESTTTNYLIELIDRVDDIYRN...,"[96, 126, 127, 128, 129, 130, 131, 132, 133, 1...",CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,3CKI_0_B,CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,"[0, 1, 2, 3, 4, 10, 65, 66, 67, 94, 95]",CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,False,256,121,3CKI_0_A_3CKI_0_B
24697,7MHY_1,7MHY_1_M,QVQLRQSGAELAKPGASVKMSCKASGYTFTNYWLHWIKQRPGQGLE...,"[43, 44, 45, 46, 60, 103, 104, 105, 106, 108, ...",DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,7MHY_1_N,DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,"[35, 44, 45, 46, 47, 50, 92, 93, 96, 97, 98, 9...",DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,False,118,109,7MHY_1_M_7MHY_1_N
24698,7MHY_2,7MHY_2_O,IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,"[36, 37, 38, 39, 53, 86, 87, 89, 90, 92, 93, 9...",IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,7MHY_2_P,VLMTQTPLSLPVSISCRSSQSIVHSNGNTYLEWYLQKPGQSPKLLI...,"[29, 39, 40, 41, 43, 46, 47, 81, 82, 83, 84, 8...",IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,False,100,94,7MHY_2_O_7MHY_2_P


In [15]:
Df_test

,interface_id,ID1,seq_target,inter_res_target,inter_seq_target,ID2,seq_binder,inter_res_binder,inter_seq_binder,dimer,seq_target_len,seq_binder_len,target_binder_id
174,1NNW_0,1NNW_0_A,VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,"[117, 118, 119, 120, 121, 140, 141, 142, 143, ...",VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,1NNW_0_B,VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,"[117, 118, 119, 120, 121, 140, 141, 142, 161, ...",VYVAVLANIAGNLPALTAALSRIEEMREEGYEIEKYYILGNIVGLF...,True,251,251,1NNW_0_A_1NNW_0_B
207,3UCN_0,3UCN_0_A,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,"[1, 2, 6, 7, 9, 10, 11, 13, 14, 25, 26, 27, 28...",TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,3UCN_0_B,TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,"[1, 2, 6, 7, 9, 10, 11, 13, 14, 25, 26, 27, 28...",TADLSPLLEANRKWADECAAKDSTYFSKVAGSQAPEYLYIGCADSR...,True,222,222,3UCN_0_A_3UCN_0_B
263,1POV_1,1POV_1_1,QHRSRSESSIESFFARGACVTIMTVDNPASTTNKDKLFAVWKITYK...,"[2, 3, 4, 6, 7, 8, 9, 12, 14, 15, 16, 48, 49, ...",GLPVMNTPGSNQYLTADNFQSPCALPEFDVTPPIDIPGEVKNMMEL...,1POV_1_3,GLPVMNTPGSNQYLTADNFQSPCALPEFDVTPPIDIPGEVKNMMEL...,"[10, 11, 12, 14, 15, 20, 21, 22, 23, 24, 27, 2...",GLPVMNTPGSNQYLTADNFQSPCALPEFDVTPPIDIPGEVKNMMEL...,False,235,238,1POV_1_1_1POV_1_3
472,3R6Y_2,3R6Y_2_C,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,"[180, 181, 182, 183, 184, 187, 190, 194, 195, ...",VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,3R6Y_2_D,VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,"[180, 181, 182, 183, 184, 188, 190, 194, 198, ...",VRIEKDFLGEKEIPKDAYYGVQTIRATENFPITGYRIHPELIKSLG...,True,383,390,3R6Y_2_C_3R6Y_2_D
585,5YHI_0,5YHI_0_A,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,"[126, 127, 143, 144, 166, 169, 170, 171, 172, ...",PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,5YHI_0_B,PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,"[126, 127, 143, 144, 166, 169, 170, 171, 172, ...",PMRYPVDVYTGKIQVDGELMLTELGLEGDGPDRALCHYPREHYLYW...,True,202,201,5YHI_0_A_5YHI_0_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24428,3GXE_0,3GXE_0_B,DQCIVDDITYNVQDTFHKKHEEGHMLNCTCFGQGRGRWKCDPVDQC...,"[2, 25, 34, 35, 36, 37, 38, 39, 40, 41]",DQCIVDDITYNVQDTFHKKHEEGHMLNCTCFGQGRGRWKCDPVDQC...,3GXE_0_F,GLGMKGHRGF,"[1, 2, 3, 4, 5, 6, 7]",DQCIVDDITYNVQDTFHKKHEEGHMLNCTCFGQGRGRWKCDPVDQC...,False,89,10,3GXE_0_B_3GXE_0_F
24450,6LY5_21,6LY5_21_d,PSPIFGGSTGGWLRKAQVEEKYVITWDSPKEQIFEMPTGGAAIMRE...,"[4, 5, 6, 7, 8, 9, 10, 11, 12, 50, 51, 52, 113]",ANFIKPYNDDPFVGHLATPITSSAVTRSLLKNLPAYRFGLTPLLRG...,6LY5_21_l,ANFIKPYNDDPFVGHLATPITSSAVTRSLLKNLPAYRFGLTPLLRG...,"[4, 5, 10, 11, 12, 13, 14, 15]",ANFIKPYNDDPFVGHLATPITSSAVTRSLLKNLPAYRFGLTPLLRG...,False,132,144,6LY5_21_d_6LY5_21_l
24451,5MLK_0,5MLK_0_A,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,"[16, 19, 22, 23, 24, 300, 301, 326, 356, 357, ...",ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,5MLK_0_B,ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,"[16, 19, 22, 23, 24, 236, 237, 262, 292, 293, ...",ARISKVLVANRGEIAVRVIRAARDAGLPSVAVYAEPDAESPHVRLA...,True,451,384,5MLK_0_A_5MLK_0_B
24543,8BS4_0,8BS4_0_A,HPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYSI...,"[76, 77, 78, 102, 103, 104, 164, 165, 170, 177...",GHPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYS...,8BS4_0_B,GHPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYS...,"[33, 77, 78, 98, 99, 100, 119, 163, 164, 176, ...",GHPVLEKLKAAHSYNPKEFEWNLKSGRVFIIKSYSEDDIHRSIKYS...,True,195,193,8BS4_0_A_8BS4_0_B


In [16]:
# Df_train.to_csv('/work3/s232958/data/PPint_DB/PPint_train_FULL.csv', index=False)
# Df_test.to_csv('/work3/s232958/data/PPint_DB/PPint_test_FULL.csv', index=False)

Df_train.to_csv('/work3/s232958/data/PPint_DB/PPint_train.csv', index=False)
Df_test.to_csv('/work3/s232958/data/PPint_DB/PPint_test.csv', index=False)

In [17]:
TRAIN_interaction_df = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_train.csv",index_col=0).reset_index(drop=True)
TEST_interaction_df = pd.read_csv("/work3/s232958/data/PPint_DB/PPint_test.csv",index_col=0).reset_index(drop=True)
TRAIN_interaction_df

,ID1,seq_target,inter_res_target,inter_seq_target,ID2,seq_binder,inter_res_binder,inter_seq_binder,dimer,seq_target_len,seq_binder_len,target_binder_id
0,6IDB_0_A,DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 16, 17, 18,...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,6IDB_0_B,GLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAID...,"[5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, ...",DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGK...,False,317,172,6IDB_0_A_6IDB_0_B
1,2WZP_3_D,VQLQESGGGLVQAGGSLRLSCTASRRTGSNWCMGWFRQLAGKEPEL...,"[37, 38, 39, 40, 41, 86, 87, 89, 90, 91, 116, ...",TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,2WZP_3_G,TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,"[164, 165, 166, 167, 203, 204, 205, 206, 207, ...",TIKNFTFFSPNSTEFPVGSNNDGKLYMMLTGMDYRTIRRKDWSSPL...,False,122,266,2WZP_3_D_2WZP_3_G
2,1ZKP_0_A,LYFQSNAKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLA...,"[15, 16, 17, 18, 19, 20, 21, 22, 41, 42, 44, 4...",AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,1ZKP_0_C,AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,"[9, 10, 11, 12, 13, 14, 15, 16, 35, 36, 38, 39...",AKTVVGFWGGFPEAGEATSGYLFEHDGFRLLVDCGSGVLAQLQKYI...,True,246,240,1ZKP_0_A_1ZKP_0_C
3,6GRH_3_C,SKHELSLVEVTHYTDPEVLAIVKDFHVRGNFASLPEFAERTFVSAV...,"[3, 5, 6, 7, 10, 11, 13, 14, 15, 246, 247, 248...",MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,6GRH_3_D,MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,"[0, 1, 2, 16, 18, 19, 20, 21, 22, 23, 24, 270,...",MINVYSNLMSAWPATMAMSPKLNRNMPTFSQIWDYERITPASAAGE...,False,266,396,6GRH_3_C_6GRH_3_D
4,8R57_1_M,DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,"[22, 23, 26, 27, 30, 31, 47, 48, 50, 51, 54, 5...",DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,8R57_1_f,PKKQKHKHKKVKLAVLQFYKVDDATGKVTRLRKECPNADCGAGTFM...,"[12, 13, 14, 15, 18, 20, 25, 26, 27, 40, 41, 4...",DLMTALQLVMKKSSAHDGLVKGLREAAKAIEKHAAQICVLAEDCDQ...,False,118,64,8R57_1_M_8R57_1_f
...,...,...,...,...,...,...,...,...,...,...,...,...
1973,4YO8_0_A,HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,"[38, 39, 58, 59, 61, 62, 65, 66, 69, 70, 73, 7...",HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,4YO8_0_B,HHHHHENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNV...,"[42, 43, 62, 63, 65, 66, 69, 70, 73, 74, 77, 7...",HENLYFQGVQKIGILGAMREEITPILELFGVDFEEIPLGGNVFHKG...,True,238,242,4YO8_0_A_4YO8_0_B
1974,3CKI_0_A,DPMKNTCKLLVVADHRFYRYMGRGEESTTTNYLIELIDRVDDIYRN...,"[96, 126, 127, 128, 129, 130, 131, 132, 133, 1...",CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,3CKI_0_B,CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,"[0, 1, 2, 3, 4, 10, 65, 66, 67, 94, 95]",CTCSPSHPQDAFCNSDIVIRAKVVGKKLVKEGPFGTLVYTIKQMKM...,False,256,121,3CKI_0_A_3CKI_0_B
1975,7MHY_1_M,QVQLRQSGAELAKPGASVKMSCKASGYTFTNYWLHWIKQRPGQGLE...,"[43, 44, 45, 46, 60, 103, 104, 105, 106, 108, ...",DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,7MHY_1_N,DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,"[35, 44, 45, 46, 47, 50, 92, 93, 96, 97, 98, 9...",DVLMTQTPLSLPVSLGDQVSISCRSSQSIVHNTYLEWYLQKPGQSP...,False,118,109,7MHY_1_M_7MHY_1_N
1976,7MHY_2_O,IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,"[36, 37, 38, 39, 53, 86, 87, 89, 90, 92, 93, 9...",IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,7MHY_2_P,VLMTQTPLSLPVSISCRSSQSIVHSNGNTYLEWYLQKPGQSPKLLI...,"[29, 39, 40, 41, 43, 46, 47, 81, 82, 83, 84, 8...",IQLVQSGPELVKISCKASGYTFTNYGMNWVRQAPGKGLKWMGWINT...,False,100,94,7MHY_2_O_7MHY_2_P


In [12]:
interaction_Dict = {}

for _, row in TRAIN_interaction_df.iterrows():
    key1 = row['ID1'].split("_")[0]+"_"+row['ID1'].split("_")[-1]
    key2 = row['ID2'].split("_")[0]+"_"+row['ID2'].split("_")[-1]
    seq1 = row['seq_target']
    seq2 = row['seq_binder']       
    interaction_Dict[key1] = seq1
    interaction_Dict[key2] = seq2

for _, row in TEST_interaction_df.iterrows():
    key1 = row['ID1'].split("_")[0]+"_"+row['ID1'].split("_")[-1]
    key2 = row['ID2'].split("_")[0]+"_"+row['ID2'].split("_")[-1]
    seq1 = row['seq_target']
    seq2 = row['seq_binder']       
    interaction_Dict[key1] = seq1
    interaction_Dict[key2] = seq2
    
list(interaction_Dict.items())[:2]

[('6IDB_A',
  'DKICLGHHAVSNGTKVNTLTERGVEVVNATETVERTNIPRICSKGKRTVDLGQCGLLGTITGPPQCDQFLEFSADLIIERREGSDVCYPGKFVNEEALRQILRESGGIDKEAMGFTYSGIRTNGATSSCRRSGSSFYAEMKWLLSNTDNAAFPQMTKSYKNTRKSPALIVWGIHHSVSTAEQTKLYGSGNKLVTVGSSNYQQSFVPSPGARTQVNGQSGRIDFHWLMLNPNDTVTFSFNGAFIAPDRASFLRGKSMGIQSGVQVDANCEGDCYHSGGTIISNLPFQNIDSRAVGKCPRYVKQRSLLLATGMKNVPEI'),
 ('6IDB_B',
  'GLFGAIAGFIENGWEGLIDGWYGFRHQNAQGEGTAADYKSTQSAIDQITGKLNRLIEKTNQQFELIDNEFNEVEKQIGNVINWTRDSITEVWSYNAELLVAMENQHTIDLADSEMDKLYERVKRQLRENAEEDGTGCFEIFHKCDDDCMASIRNNTYDHSKYREEAMQNRIQ')]

In [ ]:
### ESM-C embeddings calculated running cal_esmC.py dataset = "PPint"

# client = ESMC.from_pretrained("esmc_600m", device=torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# def calculate_ESM_pr_res_embeddings(sequence):
#     protein = ESMProtein(sequence=sequence)
#     protein_tensor = client.encode(protein)
#     logits_output = client.logits(
#     protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
#     )
#     return logits_output.embeddings.detach().cpu().numpy()

# path_to_output_embeddings = "/work3/s232958/data/PPint_DB/embeddings_esmC"
# os.makedirs(path_to_output_embeddings, exist_ok=True)

# # helper: convert torch tensor to numpy
# def to_numpy(x):
#     try:
#         return x.detach().cpu().numpy()
#     except AttributeError:
#         return np.asarray(x)
    
# for name, seq in tqdm(interaction_Dict.items(), total=len(interaction_Dict.items()), desc="Embedding targets"):
#     emb = calculate_ESM_pr_res_embeddings(seq)
#     emb_np = to_numpy(emb)
#     out_path = os.path.join(path_to_output_embeddings, f"{name}.npy")
#     np.save(out_path, emb_np)
#     print(f"Protein {name} embedded and saved to {out_path}")